# Welcome to Mellea (Docs: Welcome, Quickstart)

Welcome to the interactive demo for **Mellea**, a library for writing generative programs. 
In this notebook, we will explore how Mellea allows you to write programs that generate content, validate it against requirements, and repair it if necessary.

**Docs Reference:** [Welcome](https://docs.mellea.ai/overview/mellea-welcome), [Quickstart](https://docs.mellea.ai/overview/quick-start)

In [3]:
import mellea
from mellea.backends import ModelOption
from mellea.backends.ollama import OllamaModelBackend

# Initialize Mellea with the local Ollama backend and Granite model
print("Initializing Mellea Session...")
m = mellea.MelleaSession(
    backend=OllamaModelBackend(
        model_id="granite4:micro", 
        model_options={ModelOption.SEED: 42} # Fixed seed for reproducibility
    )
)
print("Session started! Model: granite4:micro")

Initializing Mellea Session...
Session started! Model: granite4:micro


## Your First Instruction

The core of Mellea is the `instruct` method. You give it a natural language instruction, and it returns a generated result.
Let's start with something simple.

In [4]:
response = m.instruct("Write a short haiku about writing python code.")
print(response)

  0%|          | 0/2 [00:00<?, ?it/s]

=== 16:34:05-INFO ======
SUCCESS


  0%|          | 0/2 [00:06<?, ?it/s]

Code in Python flows,
Elegance meets simplicity.
Logic takes its stage.


# The Loop: Instruct-Validate-Repair (Docs: Requirements)

Generative models can be unpredictable. Mellea's power lies in its **Instruct-Validate-Repair** loop. 
You can define **Requirements** (`req`) and **Checks** (`check`) that the output *must* satisfy. If the model fails, Mellea uses a **Strategy** to try again or fix it.

**Docs Reference:** [Requirements](https://docs.mellea.ai/overview/requirements)

### Example: Generating a Helper Email with Constraints
We want to generate an email that:
1.  Has a salutation.
2.  Does **not** use the word 'regards'.
3.  (Optional strict check) Uses only lower-case letters (just to show validation failure and repair).

In [5]:
from mellea.stdlib.requirements import req, check, simple_validate
from mellea.stdlib.sampling import RejectionSamplingStrategy

# Define requirements
requirements = [
    req("The email should have a salutation"),
    check("The email should not mention the word 'regards'"),
    # req("Use only lower-case letters", validation_fn=simple_validate(lambda s: s.islower())) # Uncomment to test strict validation failure
]

def write_email(name, notes):
    print(f"Generating email for {name}...")
    email_candidate = m.instruct(
        "Write an email to {{name}} using the notes following: {{notes}}.",
        requirements=requirements,
        # RejectionSamplingStrategy tries up to 'loop_budget' times to get a result that passes requirements
        strategy=RejectionSamplingStrategy(loop_budget=5),
        user_variables={"name": name, "notes": notes},
        return_sampling_results=True,
    )
    
    if email_candidate.success:
        print("\nSuccess! Email generated:")
        return str(email_candidate.result)
    else:
        print("\nFailed to meet requirements.")
        return email_candidate.sampling_results[0].value

# Test it
print(write_email("Arush", "Thanks for helping with the team project."))

Generating email for Arush...


  0%|          | 0/5 [00:00<?, ?it/s]

=== 16:34:11-INFO ======
SUCCESS


  0%|          | 0/5 [00:05<?, ?it/s]


Success! Email generated:
Subject: Heartfelt Thanks for Your Contribution

Dear Arush,

I hope this message finds you well. I am writing to express my sincere gratitude for your invaluable contribution towards our recent team project.

Your dedication, creativity and commitment did not go unnoticed, and they played an instrumental role in achieving the successful outcome of the project. The quality of work produced was truly impressive and a testament to your skills and abilities.

Please allow me to reiterate how much we appreciate your hard work and all that you have contributed. Your efforts have been crucial in making this project a success.

Thank you once again for everything.

Best regards,

[Your Name]


In [ ]:
import mellea

from mellea.stdlib.sampling import RejectionSamplingStrategy
from mellea.stdlib.requirements import req,check,simple_validate

# using a validation fn to validate your results as well as a check to validate your results
requirements = [
    req("The email should have a salutation"),
    req("Use only lower-case letters", validation_fn=simple_validate(lambda s: s.islower())),
    check("The email should not mention the word 'regards'")
        
]

def write_email(m:mellea.MelleaSession, names :str , notes:str)->str:
    email_candidate = m.instruct(
        "Write an email to {{name}} using the notes following: {{notes}}.",
        requirements=requirements,
        strategy = RejectionSamplingStrategy(loop_budget = 5),
        user_variables={"name": names, "notes": notes},
        return_sampling_results = True ,
    )
    if email_candidate.success :
        return str(email_candidate.result)
    else :
        print("Expect sub-par output due to failure to meet requirements.")
        return email_candidate.sampling_results[0].value
    
m=mellea.start_session()
print(write_email(m,"Arush","Arush has been a great team player, always willing to help others and contribute to group projects."))





=== 17:33:03-INFO ======
Starting Mellea session: backend=ollama, model=granite4:micro, context=SimpleContext


  0%|          | 0/5 [00:00<?, ?it/s]

=== 17:33:13-INFO ======
FAILED. Valid: 2/3


 20%|██        | 1/5 [00:09<00:38,  9.58s/it]

=== 17:33:19-INFO ======
SUCCESS


 20%|██        | 1/5 [00:15<01:03, 15.89s/it]

subject: recognition for your contributions

dear arush,

i hope this message finds you well. i am writing to express my gratitude for all that you contribute to our team environment. it has been truly inspiring to witness your dedication and commitment in every project we undertake.

your willingness to help others and your active contribution to group projects have not gone unnoticed. these qualities make you an invaluable member of our team, and they highlight the importance of being a great team player.

thank you once again for your hard work and positive attitude towards achieving our shared goals. keep up the fantastic work!

warm regards,

[your name]


# Advanced: Core Concepts & Generative Programming

**Docs Reference:** [Generative Programming](https://docs.mellea.ai/overview/project-mellea), [Core Concepts](https://docs.mellea.ai/core-concept/generative-slots)

In this section, we touch on more advanced concepts. Mellea treats prompts not just as strings, but as programs with **Slots**, **Context**, and **Agents**.

### Variable Injection (Context)
You saw `user_variables` above. This is part of Mellea's Context Management system, allowing you to inject data safely into prompts.

In [6]:
# Quick example of dynamic variables
response = m.instruct(
    "Translate the following word to Spanish: {{word}}",
    user_variables={"word": "Hello"}
)
print(f"Hello in Spanish: {response}")

  0%|          | 0/2 [00:00<?, ?it/s]

=== 16:34:11-INFO ======
SUCCESS


  0%|          | 0/2 [00:00<?, ?it/s]

Hello in Spanish: The translation of "Hello" in Spanish is "Hola".


In [7]:
# Enter your instruction below
my_instruction = "Explain quantum computing to a 5 year old."

result = m.instruct(my_instruction)
print(result)

  0%|          | 0/2 [00:00<?, ?it/s]

=== 16:34:18-INFO ======
SUCCESS


  0%|          | 0/2 [00:06<?, ?it/s]

Sure! Imagine you have a big box of toys. Now, when you want to play with your favorite toy car, in the regular games or world (which we can call 'classical computing'), it's like saying "Okay, let's only look for this car right now". You don't find other cars, you just search one by one until you find it.

Now, quantum computing is more like having a magic box of toys. In this magical box, your toy car and all the other cars can be in many places at once! It's kind of like your car being both here and there at the same time. This makes it much faster to find what you want because everything isn't stuck just in one place.

So, quantum computing is a bit different from our regular games or world - it's more magical and can do things really fast that we can't do otherwise!


In [23]:
# using model options
import mellea
from mellea.backends import ModelOption  #its mellea.backends
from mellea.backends.ollama import OllamaModelBackend
from mellea.backends import model_ids

m = mellea.MelleaSession(
    backend=OllamaModelBackend(
        model_id="granite4:micro",model_options={ModelOption.SEED: 42}
    )
)

answer = m.instruct(
    "What is 2x2?",
    model_options={
        "temperature": 0.1,
    },
)

print(str(answer))



  0%|          | 0/2 [00:00<?, ?it/s]

=== 17:33:51-INFO ======
SUCCESS


  0%|          | 0/2 [00:04<?, ?it/s]

The mathematical expression "2x2" represents the multiplication of two numbers: 2 and 2. Therefore, 2 multiplied by 2 equals 4. In other words, if you have two groups with two items in each group, the total number of items would be four.


## Generative Slots

``GenerativeSlot`` is a function whose implementation is provieded by an LLM.In Mellea you define these using ``@generative`` decorator


In [12]:
from typing import Literal
from mellea import generative, start_session


@generative
def classify_sentiment(text: str) -> Literal["positive", "negative", "neutral"]:
    """Classify the sentiment of the input text as 'positive', 'negative', or 'neutral'."""
    ...
 
m = start_session()
sentiment = classify_sentiment(m, text = "I love this product!")
print(sentiment)
    

=== 17:13:13-INFO ======
Starting Mellea session: backend=ollama, model=granite4:micro, context=SimpleContext
positive


In [21]:
# from mellea.std.base import ChatContext is old 
from mellea.stdlib.context import ChatContext
from mellea import start_session
from mellea.backends import ModelOption
from mellea import generative   

# ChatContext is used to maintain conversation history across multiple model calls. 
# Unlike SimpleContext (the default), which resets the chat history on each call, 
# ChatContext behaves like a chat history where previous messages are remembered.

@generative
def grade_syntax(code: str) -> int:
    """ Grade the code based on correct implementation of the function
    args:
        code: str (to be graded)
    returns:
        int : a grade between 1(worst) and 10(best)
    """
codes = (
    " def add(a, b):\n    return a + b",
    "def subtract(a,b): cout<<a-b<<endl",
    "int multiply(int a,int b) { return a*b}"
)
m = start_session(ctx = ChatContext() , model_options = {ModelOption.MAX_NEW_TOKENS: 10})
for code in codes :
    grade = grade_syntax(m, code = code)
    print("grade = ",grade)


=== 17:31:27-INFO ======
Starting Mellea session: backend=ollama, model=granite4:micro, context=ChatContext, model_options={'@@@max_new_tokens@@@': 10}
grade =  9
grade =  2
grade =  7


### Advanced: Using Generative slots to Provide Compositionality Across Module Boundaries

In [24]:
# file: https://github.com/generative-computing/mellea/blob/main/docs/examples/tutorial/compositionality_with_generative_slots.py#L1-L18
from mellea import generative
from typing import Literal

# The Summarizer Library
@generative
def summarize_meeting(transcript: str) -> str:
  """Summarize the meeting transcript into a concise paragraph of main points."""

@generative
def summarize_contract(contract_text: str) -> str:
  """Produce a natural language summary of contract obligations and risks."""

@generative
def summarize_short_story(story: str) -> str:
  """Summarize a short story, with one paragraph on plot and one paragraph on broad themes."""


# The Decision Aides Library
@generative
def propose_business_decision(summary: str) -> str:
  """Given a structured summary with clear recommendations, propose a business decision."""

@generative
def generate_risk_mitigation(summary: str) -> str:
  """If the summary contains risk elements, propose mitigation strategies."""

@generative
def generate_novel_recommendations(summary: str) -> str:
  """Provide a list of novel recommendations that are similar in plot or theme to the short story summary."""

# Compose the libraries.
@generative
def has_structured_conclusion(summary: str) -> Literal["yes", "no"]:
  """Determine whether the summary contains a clearly marked conclusion or recommendation."""

@generative
def contains_actionable_risks(summary: str) -> Literal["yes", "no"]:
  """Check whether the summary contains references to business risks or exposure."""

@generative
def has_theme_and_plot(summary: str) -> Literal["yes", "no"]:
  """Check whether the summary contains both a plot and thematic elements."""



In [25]:
m = start_session()
transcript = """Meeting Transcript: Market Risk Review -- Self-Sealing Stembolts Division
Date: December 1, 3125
Attendees:

Karen Rojas, VP of Product Strategy

Derek Madsen, Director of Global Procurement

Felicia Zheng, Head of Market Research

Tom Vega, CFO

Luis Tran, Engineering Liaison

Karen Rojas:
Thanks, everyone, for making time on short notice. As you've all seen, we've got three converging market risks we need to address: tariffs on micro-carburetors, increased adoption of the self-interlocking leafscrew, and, believe it or not, the "hipsterfication" of the construction industry. I need all on deck and let's not waste time. Derek, start.

Derek Madsen:
Right. As of Monday, the 25% tariff on micro-carburetors sourced from the Pan-Alpha Centauri confederacy is active. We tried to pre-purchase a three-month buffer, but after that, our unit cost rises by $1.72. That's a 9% increase in the BOM cost of our core model 440 stembolt. Unless we find alternative suppliers or pass on the cost, we're eating into our already narrow margin.

Tom Vega:
We cannot absorb that without consequences. If we pass the cost downstream, we risk losing key mid-tier OEM clients. And with the market already sniffing around leafscrew alternatives, this makes us more vulnerable.

Karen:
Lets pause there. Felicia, give us the quick-and-dirty on the leafscrew.

Felicia Zheng:
It's ugly. Sales of the self-interlocking leafscrew—particularly in modular and prefab construction—are up 38% year-over-year. It's not quite a full substitute for our self-sealing stembolts, but they are close enough in function that some contractors are making the switch. Their appeal? No micro-carburetors, lower unit complexity, and easier training for install crews. We estimate we've lost about 12% of our industrial segment to the switch in the last two quarters.

Karen:
Engineering, Luis; your take on how real that risk is?

Luis Tran:
Technically, leafscrews are not as robust under high-vibration loads. But here's the thing: most of the modular prefab sites don not need that level of tolerance. If the design spec calls for durability over 10 years, we win. But for projects looking to move fast and hit 5-year lifespans? The leafscrew wins on simplicity and cost.

Tom:
So they're eating into our low-end. That's our volume base.

Karen:
Exactly. Now let's talk about this last one: the “hipsterfication” of construction. Felicia?

Felicia:
So this is wild. We're seeing a cultural shift in boutique and residential construction—especially in markets like Beckley, West Sullivan, parts of Osborne County, where clients are requesting "authentic" manual fasteners. They want hand-sealed bolts, visible threads, even mismatched patinas. It's an aesthetic thing. Function is almost secondary. Our old manual-seal line from the 3180s? People are hunting them down on auction sites.

Tom:
Well, I'm glad I don't have to live in the big cities... nothing like this would ever happen in downt-to-earth places Brooklyn, Portland, or Austin.

Luis:
We literally got a request from a design-build firm in Keough asking if we had any bolts “pre-distressed.”

Karen:
Can we spin this?

Tom:
If we keep our vintage tooling and market it right, maybe. But that's niche. It won't offset losses in industrial and prefab.

Karen:
Not yet. But we may need to reframe it as a prestige line—low volume, high margin. Okay, action items. Derek, map alternative micro-carburetor sources. Felicia, get me a forecast on leafscrew erosion by sector. Luis, feasibility of reviving manual seal production. Tom, let's scenario-plan cost pass-through vs. feature-based differentiation.

Let's reconvene next week with hard numbers. Thanks, all."""
summary = summarize_meeting(m, transcript=transcript)

if contains_actionable_risks(m, summary=summary) == "yes":
    mitigation = generate_risk_mitigation(m, summary=summary)
    print(f"Mitigation: {mitigation}")
else:
    print("Summary does not contain actionable risks.")
if has_structured_conclusion(m, summary=summary) == "yes":
    decision = propose_business_decision(m, summary=summary)
    print(f"Decision: {decision}")
else:
    print("Summary lacks a structured conclusion.")

=== 14:47:18-INFO ======
Starting Mellea session: backend=ollama, model=granite4:micro, context=SimpleContext
Mitigation: Based on the summary provided, the following risk mitigation strategies are proposed:

1. **Supplier Diversification**: Explore alternative suppliers for micro-carburetors to mitigate potential price increases due to tariffs. This could involve conducting market research and possibly establishing new supplier relationships.

2. **Market Analysis and Forecasting**: Forecast erosion of the industrial segment sales due to increased adoption of self-interlocking leafscrews. Understanding customer trends can help in strategizing product development and marketing approaches.

3. **Product Development Assessment**: Assess feasibility of reviving manual seal production, which could cater to clients seeking authentic, manual fasteners in specific markets. This could involve a cost-benefit analysis and feasibility study.

4. **Scenario Planning for Cost Pass-Through or Featur

### Mify Objects

```MObject``` storing data along side with its relevant operations(tools).This allows LLMs to interact with both tthe data and methods in a unifiied structured manner , simplifies exposinhg only specific fields and methods for LLMs to interact with.

```Mobject``` pattern also provides a way of evolvoing exsisting classical codebases into generative programs.

In [3]:
import mellea
# old was mellea.stdlib.mify 
from mellea.stdlib.components.mify import mify, MifiedProtocol
import pandas as pd
from io import StringIO

@mify(fields_include = {"table"} , template = "{{table}}")
class myDB:
    table : str = """| Store      | Sales   |
                    | ---------- | ------- |
                    | Northeast  | $250    |
                    | Southeast  | $80     |
                    | Midwest    | $420    |"""

    def transpose(self):
        pd.read_csv(
            StringIO(self.table) , 
            sep = "|" ,
            skipinitialspace= True ,
            header = 0 ,
            index_col = False 
        )

m = mellea.start_session()
db = myDB()
assert isinstance(db , MifiedProtocol)
answer = m.query( db , " What is the total sales of the east ?")
print(str(answer))


=== 16:15:47-INFO ======
Starting Mellea session: backend=ollama, model=granite4:micro, context=SimpleContext
The total sales for the East region, which includes both the Northeast and Southeast, would be:

Northeast: $250
Southeast: $80

Total Sales = $250 + $80 = $330. 

So, the total sales for the East are $330.


### How to find new updates in the code base that cause you to use older imports and throws errors ?
```
(mellea) ashoksharma@Ashoks-MacBook-Pro Mellea_IBM % cd mellea
(mellea) ashoksharma@Ashoks-MacBook-Pro mellea % find . -name "mify.py"
./mellea/stdlib/components/mify.py
./docs/examples/mify/mify.py 
```

In [2]:
# Working with documents

from mellea.stdlib.components.docs.richdocument import RichDocument
rd = RichDocument.from_document_file("https://arxiv.org/pdf/1906.04043")

In [5]:
from mellea.stdlib.components.docs.richdocument import Table
table1 : Table = rd.get_tables()[0] 
print(table1.to_markdown())

Table 1: Cross-validated results of fake-text discriminators. Distributional information yield a higher informativeness than word-features in a logistic regression.

| Feature                              | AUC         |
|--------------------------------------|-------------|
| Bag of Words                         | 0.63 ± 0.11 |
| (Test 1 - GPT-2) Average Probability | 0.71 ± 0.25 |
| (Test 2 - GPT-2) Top-K Buckets       | 0.87 ± 0.07 |
| (Test 1 - BERT) Average Probability  | 0.70 ± 0.27 |
| (Test 2 - BERT) Top-K Buckets        | 0.85 ± 0.09 |


In [ ]:
from mellea.backends.types import ModelOption
from mellea import start_session

print("Initializing Mellea Session...")
m = mellea.MelleaSession(
    backend=OllamaModelBackend(
        model_id="granite4:micro", 
        model_options={ ModelOption.SEED: 42, ModelOption.MAX_NEW_TOKENS:500 } # Fixed seed for reproducibility
    )
)
print("Session started! Model: granite4:micro")